In [1]:
from utils import authenticate_and_initialize
authenticate_and_initialize()

In [33]:
from utils import load_geojson
import ee

# Load Mine geometry
mina_geojson = load_geojson('./assets/geo/mina.geometry.geojson')
mina_geometry = ee.Geometry(mina_geojson)
print(f"Mine geometry loaded: {mina_geometry.getInfo()['type']}")

Mine geometry loaded: MultiPolygon


In [34]:
from utils import get_embeddings_collection

# Get embeddings collection for the Mine region
embeddings = get_embeddings_collection(mina_geometry)

# Set available years directly
years = list(range(2017, 2025))
print(f"Available years: {years}")


Available years: [2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024]


In [35]:
from utils import get_year_embedding

# Get embeddings for all available years, clipped to Mine region
year_embeddings = {}
for year in years:
    emb = get_year_embedding(year, embeddings)
    year_embeddings[year] = emb.clip(mina_geometry)
    print(f"Loaded embedding for {year}")

Loaded embedding for 2017
Loaded embedding for 2018
Loaded embedding for 2019
Loaded embedding for 2020
Loaded embedding for 2021
Loaded embedding for 2022
Loaded embedding for 2023
Loaded embedding for 2024


In [36]:
from utils import create_geemap, VIS_RGB_DEFAULT

# Create map centered on Mine region
Map = create_geemap(center_geometry=mina_geometry, zoom=8, clear_default_layers=False)

# Add all years to the map
for year in years:
    Map.addLayer(
        year_embeddings[year].clip(mina_geometry),
        VIS_RGB_DEFAULT,
        f"Mine {year}",
        shown=(year == years[-1])  # Show only the most recent year by default
    )

# Add Mine boundary
#Map.addLayer(mina_geometry, {'color': 'red'}, 'Mine Boundary')

Map

Map(center=[8.865478531395857, -80.65600875632403], controls=(WidgetControl(options=['position', 'transparent_…

In [40]:
from utils import export_thumbnail, VIS_RGB_DEFAULT

# Export thumbnails for each year
urls = {}
for year in years:
    url = export_thumbnail(
        year_embeddings[year],
        mina_geometry,
        fname=f"mina_{year}",
        vis_params=VIS_RGB_DEFAULT,
        dimensions=2048,
        print_url=True
    )
    urls[year] = url

mina_2017: https://earthengine.googleapis.com/v1/projects/gen-lang-client-0315720004/thumbnails/78d4dfff8e845541db508dc948e686dd-1238cc34de47ceca4a19ec2720491e19:getPixels
mina_2018: https://earthengine.googleapis.com/v1/projects/gen-lang-client-0315720004/thumbnails/0d0007e1704427eda2ff58609ff87f32-b2d255749428336f1fa78a134929047e:getPixels
mina_2018: https://earthengine.googleapis.com/v1/projects/gen-lang-client-0315720004/thumbnails/0d0007e1704427eda2ff58609ff87f32-b2d255749428336f1fa78a134929047e:getPixels
mina_2019: https://earthengine.googleapis.com/v1/projects/gen-lang-client-0315720004/thumbnails/b185dc3b0fadf88719a0f85a40b0053b-a5dff39a93cb8a5111b14f891da8ecc4:getPixels
mina_2019: https://earthengine.googleapis.com/v1/projects/gen-lang-client-0315720004/thumbnails/b185dc3b0fadf88719a0f85a40b0053b-a5dff39a93cb8a5111b14f891da8ecc4:getPixels
mina_2020: https://earthengine.googleapis.com/v1/projects/gen-lang-client-0315720004/thumbnails/a411447edeca2306841aeea8c14a1bca-734ff6ede9e

In [38]:
# Display all thumbnails side by side
from IPython.display import Image, HTML, display

html_content = '<div style="display: flex; flex-wrap: wrap; gap: 10px;">'
for year in years:
    html_content += f'''
    <div style="text-align: center;">
        <h3>{year}</h3>
        <img src="{urls[year]}" width="400" />
    </div>
    '''
html_content += '</div>'

display(HTML(html_content))